Code for making: https://www.kaggle.com/thunderz/a-month-of-askreddit

In [ ]:
import sys
import os

sys.path.insert(0, os.getcwd() + '/reddit_download')

In [ ]:
from reddit_download.RWV.pushshift.time_utils import timestamp_to_utc
from reddit_download.RWV.pushshift.utils import build_df, apply_df_time_transforms

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
df_comments = build_df(content_type='comment',
                       file_path=os.getcwd() + '/reddit_download',
                       subreddits=['reddit_data_askreddit/AskReddit'],
                       workers=12)

df_posts = build_df(content_type='post',
                    file_path=os.getcwd() + '/reddit_download',
                    subreddits=['reddit_data_askreddit/AskReddit'],
                    workers=12)

In [ ]:
df_comments.shape

In [ ]:
df_posts.shape

In [ ]:
df_comments = df_comments.rename(columns={"link_id": "post_id"})

df_comments = df_comments.rename(columns={"created_utc": "timestamp"})
df_posts = df_posts.rename(columns={"created_utc": "timestamp"})

In [ ]:
ind = df_comments[df_comments['author'] == '[deleted]'].index
df_comments.drop(ind, inplace=True)

ind = df_comments[df_comments['author'] == 'AutoModerator'].index
df_comments.drop(ind, inplace=True)

ind = df_posts[df_posts['author'] == '[deleted]'].index
df_posts.drop(ind, inplace=True)

ind = df_posts[df_posts['author'] == 'AutoModerator'].index
df_posts.drop(ind, inplace=True)

In [ ]:
post_id = df_comments['post_id'].copy()

df_comments['post_id'] = df_comments['post_id'].apply(lambda x: x.split('_')[1])

In [ ]:
k = df_posts['post_id']# , df_posts['timestamp']
v = np.ones(len(k), dtype=int)
id_to_timestamp = dict(zip(k, v))

import swifter

def func(x, mapping):
    try:
        return mapping[x]
    except KeyError:
        return -1

df_comments['has_post'] = df_comments['post_id'].swifter.apply(func, args=(id_to_timestamp, ))

In [ ]:
ind = df_comments[df_comments['has_post'] == 1].index
post_id = post_id[ind]

In [ ]:
ind = df_comments[df_comments['has_post'] == -1].index
df_comments.drop(ind, inplace=True)

In [ ]:
df_comments['post_id'] = post_id

In [ ]:
del df_comments['has_post']

In [ ]:
df_comments.reset_index(inplace=True)
df_posts.reset_index(inplace=True)

In [ ]:
del df_comments['index']

In [ ]:
del df_posts['index']

In [ ]:
del df_comments['subreddit']
del df_posts['subreddit']

In [ ]:
print(timestamp_to_utc(df_comments['timestamp'].max()))

In [ ]:
print(timestamp_to_utc(df_comments['timestamp'].min()))

In [ ]:
df_comments.sort_values(by=['timestamp'], inplace=True)

In [ ]:
df_posts.sort_values(by=['timestamp'], inplace=True)

In [ ]:
# df_comments.to_csv('askreddit_comments.csv', encoding='utf-8', index=False)

In [ ]:
# df_posts.to_csv('askreddit_posts.csv', encoding='utf-8', index=False)